# Term Project Spring 2020 - Team Game Cancelled
___

## Course Data Warehouse Test

<b> Table of Contents: </b>
<br> [1. Star Schema](#000)
<br> [2. Set Up - SQL and Database Warehouse](#100)
<br> [3. Creating Tables from Star Schema](#200)
<br> [4. Load Data into the Tables ](#300)
___

<a id = "000"> <h2> 1. Star Schema </h2> </a>
_Design and build data warehouse called CourseDataWarehouse.db._

![Star Schema](James/images/StarSchema.png)

<a id = "100"> <h2> 2. Set Up - SQL and Database Warehouse </h2> </a>

### Imports

In [1]:
%load_ext sql
import pandas as pd
import sqlite3

In [2]:
# Do we still need this part?
%sql sqlite:///CourseDataWarehouse.db
conn = sqlite3.connect('CourseDataWarehouse.db')

In [3]:
%%sql
ATTACH DATABASE 'CourseData.db' AS CourseData;

 * sqlite:///CourseDataWarehouse.db
Done.


[]

<a id = "200"> <h2> 3. Create Tables from Star Schema </h2> </a>

In [4]:
%%sql

-- Instructors Dimension table
DROP TABLE IF EXISTS INSTRUCTORS_DIM;
CREATE TABLE INSTRUCTORS_DIM (
    InstructorID INTEGER PRIMARY KEY,
    Name TEXT
);

-- Location Dimension table
DROP TABLE IF EXISTS LOCATION_DIM;
CREATE TABLE LOCATION_DIM (
    LocationID INTEGER PRIMARY KEY,
    Location TEXT
);

-- Timecode Dimension table
DROP TABLE IF EXISTS TIMECODE_DIM;
CREATE TABLE TIMECODE_DIM (
    TimecodeID INTEGER PRIMARY KEY,
    Day TEXT,
    Start TEXT,
    End Text
);

-- Programs Dimension table
DROP TABLE IF EXISTS PROGRAMS_DIM;
CREATE TABLE PROGRAMS_DIM (
    ProgramID INTEGER PRIMARY KEY,
    ProgramCode TEXT,
    ProgramName TEXT
);

-- Terms Dimension table
DROP TABLE IF EXISTS TERMS_DIM;
CREATE TABLE TERMS_DIM (
    TermsID INTEGER PRIMARY KEY,
    Term TEXT
);

-- Course Catalogs Dimension table
DROP TABLE IF EXISTS COURSE_CATALOGS_DIM;
CREATE TABLE COURSE_CATALOGS_DIM(
    CourseID INTEGER PRIMARY KEY,
    CatalogYear TEXT,
    CatalogID TEXT,
    CourseTitle TEXT,
    Credits TEXT
);

-- Course Meetings Fact table
DROP TABLE IF EXISTS COURSE_MEETINGS_FACT;
CREATE TABLE COURSE_MEETINGS_FACT(
    CourseMeetingID INTEGER,
    CatalogID INTEGER,
    ProgramID INTEGER,
    InstructorID INTEGER,
    CourseID INTEGER,
    CourseOfferingID INTEGER,
    TimecodeID INTEGER,
    TermsID INTEGER,
    Term TEXT,
    LocationID INTEGER,
    CRN INTEGER,
    PrimaryInstructorID TEXT,
    Capacity INTEGER,
    Actual INTEGER,
    Remaining INTEGER,
    Credits INTEGER,
    FOREIGN KEY (TimecodeID) REFERENCES TIMECODE_DIM(TimecodeID),
    FOREIGN KEY (TermsID) REFERENCES TERMS_DIM(TermsID),
    FOREIGN KEY (InstructorID) REFERENCES INSTRUCTOR_DIM(InstructorID),
    FOREIGN KEY (ProgramID) REFERENCES PROGRAMS_DIM(ProgramID),
    FOREIGN KEY (CourseID) REFERENCES COURSE_CATALOGS_DIM(CourseID),
    FOREIGN KEY (LocationID) REFERENCES LOCATION_DIM(LocationID)

    
    
);


 * sqlite:///CourseDataWarehouse.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

<a id = "300"> <h2> 3. Load Data into our Tables </h2> </a>

### Rollup Queries

_We use rollup queries to populate our tables._

In [5]:
%%sql
DELETE FROM INSTRUCTORS_DIM;

INSERT INTO INSTRUCTORS_DIM
SELECT InstructorID, Name
FROM CourseData.INSTRUCTORS;

 * sqlite:///CourseDataWarehouse.db
0 rows affected.
1095 rows affected.


[]

In [6]:
%%sql
-- Record Counts for Course Meetings
SELECT 
    (SELECT Count(*) FROM INSTRUCTORS_DIM) as 'RawCount',
    (SELECT Count(*) FROM (SELECT DISTINCT * FROM INSTRUCTORS_DIM)) as 'DistinctCount';

 * sqlite:///CourseDataWarehouse.db
Done.


RawCount,DistinctCount
1095,1095


_Testing the data loaded into Coursedata Warehouse_

In [7]:
%%sql
DELETE FROM LOCATION_DIM;

INSERT INTO LOCATION_DIM
SELECT DISTINCT LocationID, Location
FROM CourseData.LOCATION;

 * sqlite:///CourseDataWarehouse.db
0 rows affected.
207 rows affected.


[]

In [8]:
%%sql 
SELECT *
FROM LOCATION_DIM
LIMIT 5;

 * sqlite:///CourseDataWarehouse.db
Done.


LocationID,Location
1,BCC 200
2,BD
3,BH
4,BH BY ARR
5,BLM 112


In [9]:
%%sql
-- Record Counts for Course Meetings
SELECT 
    (SELECT Count(*) FROM LOCATION_DIM) as 'RawCount',
    (SELECT Count(*) FROM (SELECT DISTINCT * FROM LOCATION_DIM)) as 'DistinctCount';

 * sqlite:///CourseDataWarehouse.db
Done.


RawCount,DistinctCount
207,207


In [10]:
%%sql
DELETE FROM TIMECODE_DIM;

INSERT INTO TIMECODE_DIM(Day, Start, End)
SELECT DISTINCT Day, StartDateTime, EndDateTime
FROM CourseData.COURSE_MEETINGS;

 * sqlite:///CourseDataWarehouse.db
0 rows affected.
39298 rows affected.


[]

In [11]:
%%sql
-- Record Counts for Course Meetings
SELECT 
    (SELECT Count(*) FROM TIMECODE_DIM) as 'RawCount',
    (SELECT Count(*) FROM (SELECT DISTINCT * FROM TIMECODE_DIM)) as 'DistinctCount';

 * sqlite:///CourseDataWarehouse.db
Done.


RawCount,DistinctCount
39298,39298


In [12]:
%%sql
SELECT *
FROM TIMECODE_DIM
LIMIT 10;

 * sqlite:///CourseDataWarehouse.db
Done.


TimecodeID,Day,Start,End
1,T,2014-09-02T08:00:00,2014-09-02T09:15:00
2,F,2014-09-05T08:00:00,2014-09-05T09:15:00
3,T,2014-09-09T08:00:00,2014-09-09T09:15:00
4,F,2014-09-12T08:00:00,2014-09-12T09:15:00
5,T,2014-09-16T08:00:00,2014-09-16T09:15:00
6,F,2014-09-19T08:00:00,2014-09-19T09:15:00
7,T,2014-09-23T08:00:00,2014-09-23T09:15:00
8,F,2014-09-26T08:00:00,2014-09-26T09:15:00
9,T,2014-09-30T08:00:00,2014-09-30T09:15:00
10,F,2014-10-03T08:00:00,2014-10-03T09:15:00


In [13]:
%%sql
DELETE FROM PROGRAMS_DIM;

INSERT INTO PROGRAMS_DIM
SELECT ProgramID, ProgramCode, ProgramName
FROM CourseData.PROGRAMS; 

 * sqlite:///CourseDataWarehouse.db
0 rows affected.
83 rows affected.


[]

In [14]:
%%sql
-- Record Counts for Course Meetings
SELECT 
    (SELECT Count(*) FROM PROGRAMS) as 'RawCount',
    (SELECT Count(*) FROM (SELECT DISTINCT * FROM PROGRAMS)) as 'DistinctCount';

 * sqlite:///CourseDataWarehouse.db
Done.


RawCount,DistinctCount
83,83


In [15]:
%%sql
DELETE FROM TERMS_DIM;

INSERT INTO TERMS_DIM(Term)
SELECT DISTINCT Term
FROM CourseData.COURSE_OFFERINGS;

 * sqlite:///CourseDataWarehouse.db
0 rows affected.
19 rows affected.


[]

In [16]:
%%sql
-- Record Counts for Course Meetings
SELECT 
    (SELECT Count(*) FROM TERMS_DIM) as 'RawCount',
    (SELECT Count(*) FROM (SELECT DISTINCT * FROM TERMS_DIM)) as 'DistinctCount';

 * sqlite:///CourseDataWarehouse.db
Done.


RawCount,DistinctCount
19,19


In [17]:
%%sql
SELECT *
FROM TERMS_DIM
;

 * sqlite:///CourseDataWarehouse.db
Done.


TermsID,Term
1,Fall2014
2,Fall2015
3,Fall2016
4,Fall2017
5,Fall2018
6,Spring2015
7,Spring2016
8,Spring2017
9,Spring2018
10,Spring2019


In [18]:
%%sql
DELETE FROM COURSE_CATALOGS_DIM;

INSERT INTO COURSE_CATALOGS_DIM
SELECT CourseID, CatalogYear, CatalogID, CourseTitle, Credits 
FROM CourseData.COURSE_CATALOGS;

 * sqlite:///CourseDataWarehouse.db
0 rows affected.
4440 rows affected.


[]

In [19]:
%%sql
SELECT DISTINCT *
FROM COURSE_CATALOGS_DIM
LIMIT (5);

 * sqlite:///CourseDataWarehouse.db
Done.


CourseID,CatalogYear,CatalogID,CourseTitle,Credits
1,2017_2018,AN 0301,Independent Study,1-3 Credits
2,2017_2018,AN 0310,Asian Studies Seminar,3 Credits
3,2017_2018,BU 0211,Legal Environment of Business,3 Credits
4,2017_2018,BU 0220,Environmental Law and Policy,3 Credits
5,2017_2018,BU 0311,"The Law of Contracts, Sales, and Property",3 Credits


In [20]:
%%sql

SELECT *
FROM 
LOCATION_DIM

LIMIT 5;

 * sqlite:///CourseDataWarehouse.db
Done.


LocationID,Location
1,BCC 200
2,BD
3,BH
4,BH BY ARR
5,BLM 112


In [21]:
%%sql
--select query to see what the info will look like before we insert into the fact table

SELECT *
FROM CourseData.COURSE_MEETINGS
   JOIN CourseData.COURSE_OFFERINGS USING (CourseOfferingID)
    JOIN PROGRAMS_DIM ON (PROGRAMS_DIM.ProgramCode = SUBSTR(CourseData.COURSE_OFFERINGS.CatalogID,1,INSTR(CourseData.COURSE_OFFERINGS.CatalogID," ")-1))
    LEFT JOIN INSTRUCTORS_DIM ON (INSTRUCTORS_DIM.Name = CourseData.COURSE_OFFERINGS.PrimaryInstructorID)
    LEFT JOIN LOCATION_DIM USING (LocationID)
--      LEFT JOIN TIMECODE_DIM ON (TIMECODE_DIM.Day = CourseData.COURSE_MEETINGS.Day)

WHERE Term = 'Fall2017'
LIMIT 5;


 * sqlite:///CourseDataWarehouse.db
Done.


CourseMeetingID,CourseOfferingID,CRN,LocationID,Day,StartDateTime,EndDateTime,CourseID,Term,CRN_1,CatalogID,Section,Credits,Title,Timecodes,PrimaryInstructorID,Capacity,Actual,Remaining,ProgramID,ProgramCode,ProgramName,InstructorID,Name,Location
164495,8706,71188,105,R,2017-09-07T11:00:00,2017-09-07T12:15:00,113,Fall2017,71188,AC 0011,01C,3.000,Introduction to Financial Accounting,['MR 1100am-1215pm 09/05-12/11 DSB 110A'],Rebecca I. Bloch,0,28,-28,1,AC,Accounting,2,Rebecca I. Bloch,DSB 110A
164496,8707,71188,105,R,2017-09-07T11:00:00,2017-09-07T12:15:00,2333,Fall2017,71188,AC 0011,01C,3.000,Introduction to Financial Accounting,['MR 1100am-1215pm 09/05-12/11 DSB 110A'],Rebecca I. Bloch,0,28,-28,1,AC,Accounting,2,Rebecca I. Bloch,DSB 110A
164497,8706,71188,105,M,2017-09-11T11:00:00,2017-09-11T12:15:00,113,Fall2017,71188,AC 0011,01C,3.000,Introduction to Financial Accounting,['MR 1100am-1215pm 09/05-12/11 DSB 110A'],Rebecca I. Bloch,0,28,-28,1,AC,Accounting,2,Rebecca I. Bloch,DSB 110A
164498,8707,71188,105,M,2017-09-11T11:00:00,2017-09-11T12:15:00,2333,Fall2017,71188,AC 0011,01C,3.000,Introduction to Financial Accounting,['MR 1100am-1215pm 09/05-12/11 DSB 110A'],Rebecca I. Bloch,0,28,-28,1,AC,Accounting,2,Rebecca I. Bloch,DSB 110A
164499,8706,71188,105,R,2017-09-14T11:00:00,2017-09-14T12:15:00,113,Fall2017,71188,AC 0011,01C,3.000,Introduction to Financial Accounting,['MR 1100am-1215pm 09/05-12/11 DSB 110A'],Rebecca I. Bloch,0,28,-28,1,AC,Accounting,2,Rebecca I. Bloch,DSB 110A


__CREATING COURSE MEETINGS FACT TABLE__

In [ ]:
%%sql
DELETE FROM COURSE_MEETINGS_FACT;

INSERT INTO COURSE_MEETINGS_FACT(CourseMeetingID,CatalogID,ProgramID,InstructorID,CourseID,CourseOfferingID,TimecodeID,TermsID,Term,LocationID,CRN,PrimaryInstructorID,Capacity,Actual,Remaining,Credits)
SELECT DISTINCT CourseData.COURSE_MEETINGS.CourseMeetingID,COURSE_OFFERINGS.CatalogID, PROGRAMS_DIM.ProgramID,INSTRUCTORS_DIM.InstructorID,COURSE_OFFERINGS.CourseID, COURSE_OFFERINGS.CourseOfferingID,TIMECODE_DIM.TimecodeID,TERMS_DIM.TermsID, TERMS_DIM.Term, LOCATION_DIM.LocationID, COURSE_OFFERINGS.CRN, CourseData.COURSE_OFFERINGS.PrimaryInstructorID, COURSE_OFFERINGS.Capacity, COURSE_OFFERINGS.Actual, COURSE_OFFERINGS.Remaining, COURSE_OFFERINGS.Credits

FROM CourseData.COURSE_MEETINGS
    JOIN CourseData.COURSE_OFFERINGS USING (CourseOfferingID)
    JOIN PROGRAMS_DIM ON (PROGRAMS_DIM.ProgramCode = SUBSTR(CourseData.COURSE_OFFERINGS.CatalogID,1,2))
    LEFT JOIN INSTRUCTORS_DIM ON (INSTRUCTORS_DIM.Name = CourseData.COURSE_OFFERINGS.PrimaryInstructorID)
    LEFT JOIN LOCATION_DIM USING (LocationID)
    LEFT JOIN TIMECODE_DIM ON (TIMECODE_DIM.Day = CourseData.COURSE_MEETINGS.Day)
    LEFT JOIN TERMS_DIM USING (Term)
    LEFT JOIN COURSE_CATALOGS_DIM USING (CourseID, CatalogID)
LIMIT 164499;

In [22]:
%%sql
DELETE FROM COURSE_MEETINGS_FACT;

INSERT INTO COURSE_MEETINGS_FACT(CourseMeetingID,CatalogID,ProgramID,InstructorID,CourseID,CourseOfferingID,TimecodeID,TermsID,Term,LocationID,CRN,PrimaryInstructorID,Capacity,Actual,Remaining,Credits)
SELECT DISTINCT CourseData.COURSE_MEETINGS.CourseMeetingID,COURSE_OFFERINGS.CatalogID, PROGRAMS_DIM.ProgramID,INSTRUCTORS_DIM.InstructorID,COURSE_OFFERINGS.CourseID, COURSE_OFFERINGS.CourseOfferingID,TIMECODE_DIM.TimecodeID,TERMS_DIM.TermsID, TERMS_DIM.Term, LOCATION_DIM.LocationID, COURSE_OFFERINGS.CRN, CourseData.COURSE_OFFERINGS.PrimaryInstructorID, COURSE_OFFERINGS.Capacity, COURSE_OFFERINGS.Actual, COURSE_OFFERINGS.Remaining, COURSE_OFFERINGS.Credits

FROM CourseData.COURSE_MEETINGS
    JOIN CourseData.COURSE_OFFERINGS USING (CourseOfferingID)
    JOIN PROGRAMS_DIM ON (PROGRAMS_DIM.ProgramCode = SUBSTR(CourseData.COURSE_OFFERINGS.CatalogID,1,2))
    LEFT JOIN INSTRUCTORS_DIM ON (INSTRUCTORS_DIM.Name = CourseData.COURSE_OFFERINGS.PrimaryInstructorID)
    LEFT JOIN LOCATION_DIM USING (LocationID)
    LEFT JOIN TIMECODE_DIM ON (TIMECODE_DIM.Day = CourseData.COURSE_MEETINGS.Day)
    LEFT JOIN TERMS_DIM USING (Term)
    LEFT JOIN COURSE_CATALOGS_DIM USING (CourseID, CatalogID)
LIMIT 284847;

 * sqlite:///CourseDataWarehouse.db
0 rows affected.
284847 rows affected.


[]

In [32]:
%%sql

SELECT *
FROM COURSE_MEETINGS_FACT
LIMIT 10;

 * sqlite:///CourseDataWarehouse.db
Done.


CourseMeetingID,CatalogID,ProgramID,InstructorID,CourseID,CourseOfferingID,TimecodeID,TermsID,Term,LocationID,CRN,PrimaryInstructorID,Capacity,Actual,Remaining,Credits
1,AC 0011,1,1,113,1,1,1,Fall2014,99,70384,Michael P. Coyne,0,31,-31,3
1,AC 0011,1,1,113,1,3,1,Fall2014,99,70384,Michael P. Coyne,0,31,-31,3
1,AC 0011,1,1,113,1,5,1,Fall2014,99,70384,Michael P. Coyne,0,31,-31,3
1,AC 0011,1,1,113,1,7,1,Fall2014,99,70384,Michael P. Coyne,0,31,-31,3
1,AC 0011,1,1,113,1,9,1,Fall2014,99,70384,Michael P. Coyne,0,31,-31,3
1,AC 0011,1,1,113,1,11,1,Fall2014,99,70384,Michael P. Coyne,0,31,-31,3
1,AC 0011,1,1,113,1,14,1,Fall2014,99,70384,Michael P. Coyne,0,31,-31,3
1,AC 0011,1,1,113,1,16,1,Fall2014,99,70384,Michael P. Coyne,0,31,-31,3
1,AC 0011,1,1,113,1,18,1,Fall2014,99,70384,Michael P. Coyne,0,31,-31,3
1,AC 0011,1,1,113,1,20,1,Fall2014,99,70384,Michael P. Coyne,0,31,-31,3


In [59]:
%%sql

SELECT DISTINCT *
FROM COURSE_MEETINGS_FACT
ORDER BY RANDOM()
LIMIT 10;


 * sqlite:///CourseDataWarehouse.db
Done.


CourseMeetingID,CatalogID,ProgramID,InstructorID,CourseID,CourseOfferingID,TimecodeID,TermsID,Term,LocationID,CRN,PrimaryInstructorID,Capacity,Actual,Remaining,Credits
35,AC 0011,1,1,113,1,35572,1,Fall2014,99,70384,Michael P. Coyne,0,31,-31,3
43,AC 0011,1,1,113,1,7915,1,Fall2014,99,70384,Michael P. Coyne,0,31,-31,3
2,AC 0011,1,1,2333,2,3373,1,Fall2014,99,70384,Michael P. Coyne,0,31,-31,3
40,AC 0011,1,1,2333,2,3223,1,Fall2014,99,70384,Michael P. Coyne,0,31,-31,3
10,AC 0011,1,1,2333,2,29833,1,Fall2014,99,70384,Michael P. Coyne,0,31,-31,3
14,AC 0011,1,1,2333,2,6242,1,Fall2014,99,70384,Michael P. Coyne,0,31,-31,3
37,AC 0011,1,1,113,1,34267,1,Fall2014,99,70384,Michael P. Coyne,0,31,-31,3
36,AC 0011,1,1,2333,2,37042,1,Fall2014,99,70384,Michael P. Coyne,0,31,-31,3
17,AC 0011,1,1,113,1,50,1,Fall2014,99,70384,Michael P. Coyne,0,31,-31,3
8,AC 0011,1,1,2333,2,20279,1,Fall2014,99,70384,Michael P. Coyne,0,31,-31,3


In [ ]:
%%sql
vacuum;